In [ ]:
import sys
import os
workspace_dir = '/root/Document/ai_ct_seg/CardiacSegV2'
sys.path.append(workspace_dir)

# 確定dataset內容

In [ ]:
import os
pred_img = []
p = os.path.join(workspace_dir, "dataset/chgh/pred/")
for root, dirs, files in os.walk(p, topdown=False):
  for name in files:
    pred_img.append(os.path.join(root, name))
    print(os.path.join(root, name))

/root/Document/ai_ct_seg/CardiacSegV2/dataset/chgh/pred/patient0055.nii.gz
/root/Document/ai_ct_seg/CardiacSegV2/dataset/chgh/pred/patient0062.nii.gz
/root/Document/ai_ct_seg/CardiacSegV2/dataset/chgh/pred/patient0061.nii.gz
/root/Document/ai_ct_seg/CardiacSegV2/dataset/chgh/pred/patient0085.nii.gz
/root/Document/ai_ct_seg/CardiacSegV2/dataset/chgh/pred/patient0053.nii.gz
/root/Document/ai_ct_seg/CardiacSegV2/dataset/chgh/pred/patient0060.nii.gz
/root/Document/ai_ct_seg/CardiacSegV2/dataset/chgh/pred/patient0090.nii.gz
/root/Document/ai_ct_seg/CardiacSegV2/dataset/chgh/pred/patient0068.nii.gz
/root/Document/ai_ct_seg/CardiacSegV2/dataset/chgh/pred/patient0084.nii.gz
/root/Document/ai_ct_seg/CardiacSegV2/dataset/chgh/pred/patient0051.nii.gz
/root/Document/ai_ct_seg/CardiacSegV2/dataset/chgh/pred/patient0100.nii.gz
/root/Document/ai_ct_seg/CardiacSegV2/dataset/chgh/pred/patient0083.nii.gz
/root/Document/ai_ct_seg/CardiacSegV2/dataset/chgh/pred/patient0071.nii.gz
/root/Document/ai_ct_seg/

# 推論設定

In [ ]:
import sys
# workspace_dir = '/root/Document/ai_ct_seg/CardiacSegV2'

import os
#from expers.infer_utils import get_tune_model_dir, get_data_path, get_pred_path


# need to check model name and model path
model_name = "unetr_pp_linear_on3" #unet3d
data_name = 'chgh'
# exp_name = 'exp_swin'

# set data dir
root_data_dir = os.path.join(workspace_dir,'dataset',data_name)
#data_dir = os.path.join(root_data_dir, sub_data_dir_name)
data_dir = os.path.join(root_data_dir)

root_exp_dir = os.path.join(workspace_dir,'exps','exps',model_name,data_name,'tune_results')
model_dir = "/root/Document/ai_ct_seg/CardiacSegV2/exps/exps/unetr_pp_linear_on3/chgh/tune_results/exp_roi128_gdl_train45/main_0ab9c_00000_0_exp=exp_exp_roi128_gdl_train45_2025-11-10_13-42-28/models/" # get_tune_model_dir(root_exp_dir, exp_name)
best_checkpoint = os.path.join(model_dir, 'best_model.pth')
infer_dir = os.path.join(workspace_dir, '/myo_pred',"chgh","infer")
#

# 執行推論

In [7]:
for img_pth in pred_img:
  !python ../CardiacSegV2/expers/infer.py \
  --model_name={model_name} \
  --data_name={data_name} \
  --data_dir={data_dir} \
  --model_dir={model_dir} \
  --infer_dir={infer_dir} \
  --checkpoint={best_checkpoint} \
  --img_pth={img_pth} \
  --out_channels=4 \
  --patch_size=2 \
  --feature_size=24 \
  --drop_rate=0.1 \
  --depths 3 3 3 3 \
  --kernel_size 7 \
  --exp_rate 4 \
  --norm_name='layer' \
  --a_min=-42 \
  --a_max=423 \
  --space_x=0.7 \
  --space_y=0.7 \
  --space_z=1.0 \
  --roi_x=128 \
  --roi_y=128 \
  --roi_z=128 \
  --infer_post_process

/root/anaconda3/envs/ctseg/lib/python3.10/site-packages/monai/utils/module.py:395: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  pkg = __import__(module)  # top level module
cuda is available
model: unetr_pp_linear_on3
 Network:  Using Unetr_pp linear on decoder 3
 do ds:  None
=> loaded checkpoint '/root/Document/ai_ct_seg/CardiacSegV2/exps/exps/unetr_pp_linear_on3/chgh/tune_results/exp_roi128_gdl_train45/main_0ab9c_00000_0_exp=exp_exp_roi128_gdl_train45_2025-11-10_13-42-28/models/best_model.pth')
infer data: {'image': '/root/Document/ai_ct_seg/CardiacSegV2/dataset/chgh/pred/patient0055.nii.gz'}
monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in 

# 整理預測結果檔案

In [ ]:
import os
import zipfile
import glob

# -----------------
# 1. 定義路徑
# -----------------

# 包含要壓縮的 .nii.gz 文件的目錄 (當前目錄)
source_dir = infer_dir

# 上一層目錄的路徑
# 使用 os.path.dirname() 獲取 source_dir 的父目錄
parent_dir = os.path.dirname(source_dir)

# 最終 zip 檔案的路徑和名稱
zip_filename = 'predict.zip'
zip_filepath = os.path.join(parent_dir, zip_filename)

# -----------------
# 2. 執行壓縮
# -----------------
def create_zip_archive(source_dir, zip_filepath):
    """
    將指定目錄下的所有文件壓縮到指定的 zip 檔案中。
    
    Args:
        source_dir (str): 要壓縮的文件的來源目錄。
        zip_filepath (str): 輸出 zip 檔案的完整路徑。
    """
    
    # 檢查來源目錄是否存在
    if not os.path.isdir(source_dir):
        print(f"錯誤：來源目錄不存在: {source_dir}")
        return

    # 確保目標 zip 檔案所在的父目錄存在 (這裡不需要，因為 parent_dir 應該存在)
    # os.makedirs(os.path.dirname(zip_filepath), exist_ok=True)

    print(f"開始壓縮目錄：{source_dir}")
    print(f"目標 zip 檔案：{zip_filepath}")
    
    # 使用 zipfile.ZipFile 建立 zip 檔案
    # 模式 'w' 表示寫入 (會覆蓋已存在的同名文件)
    try:
        with zipfile.ZipFile(zip_filepath, 'w', zipfile.ZIP_DEFLATED) as zipf:
            
            # 遍歷 source_dir 中的所有文件
            for filename in os.listdir(source_dir):
                file_path = os.path.join(source_dir, filename)
                
                # 只處理文件，忽略子目錄
                if os.path.isfile(file_path):
                    
                    # 關鍵步驟：
                    # 將文件寫入 zip 檔案，並指定在 zip 內部的名稱 (arcname)。
                    # 這裡我們只使用 basename (例如: image_001_predict.nii.gz)
                    # 這樣可以避免在 zip 檔案內部創建 'root/Document/...' 這樣的目錄結構。
                    arcname = filename 
                    
                    zipf.write(file_path, arcname)
                    print(f"  -> 已加入: {filename}")

        print(f"\n✅ 壓縮完成！檔案已儲存至：{zip_filepath}")

    except Exception as e:
        print(f"❌ 壓縮過程中發生錯誤: {e}")

# 執行函數
create_zip_archive(source_dir, zip_filepath)

開始壓縮目錄：/root/Document/ai_ct_seg/myo_pred/chgh/infer
目標 zip 檔案：/root/Document/ai_ct_seg/myo_pred/chgh/predict.zip
  -> 已加入: patient0074_predict.nii.gz
  -> 已加入: patient0071_predict.nii.gz
  -> 已加入: patient0066_predict.nii.gz
  -> 已加入: patient0056_predict.nii.gz
  -> 已加入: patient0093_predict.nii.gz
  -> 已加入: patient0087_predict.nii.gz
  -> 已加入: patient0069_predict.nii.gz
  -> 已加入: patient0057_predict.nii.gz
  -> 已加入: patient0073_predict.nii.gz
  -> 已加入: patient0060_predict.nii.gz
  -> 已加入: patient0079_predict.nii.gz
  -> 已加入: patient0091_predict.nii.gz
  -> 已加入: patient0072_predict.nii.gz
  -> 已加入: patient0067_predict.nii.gz
  -> 已加入: patient0097_predict.nii.gz
  -> 已加入: patient0085_predict.nii.gz
  -> 已加入: patient0063_predict.nii.gz
  -> 已加入: patient0100_predict.nii.gz
  -> 已加入: patient0083_predict.nii.gz
  -> 已加入: patient0098_predict.nii.gz
  -> 已加入: patient0070_predict.nii.gz
  -> 已加入: patient0096_predict.nii.gz
  -> 已加入: patient0077_predict.nii.gz
  -> 已加入: patient0092_predict.nii.gz